# Selection Table Reader

This Notebook contains code which reads a list of directories containing several selection table and groups the wavefiles associated with each selection. An example of a selection table name format is `1705_20170929_231347_062.Table.1.selections.txt` and an example of a corresponding wav file that might be classified in that table is `1705_20170929_231347_sel_13.wav`.

In [ ]:
import os
import os
import numpy as np
import shutil

#TODO: ensure name standardisation in selection tables


# get slection table paths
main_dir = # path to main directory
dir_list = os.listdir(main_dir)
selection_tables = [] # initialise list of selection tables
print(os.listdir(main_dir))


for dir in dir_list:

    # ignore any Python notebooks in directory
    if dir.endswith(".ipynb"):
        continue
    
    # find all selection table file paths
    elif os.path.isdir(dir):

        for file in os.listdir(os.path.join(main_dir, dir)):

            if file.endswith(".selections.txt"):
                selection_tables.append(os.path.join(main_dir, dir, file))


removed_end_str_length = 26 # length to be removed from end of string

whistle_dir_name = "Whistles"
non_whistle_dir_name = "No Whistles"

for selection_table in selection_tables:

    # get column data from selection tables
    columns = np.genfromtxt(selection_table, dtype=str, delimiter="\t", skip_header=True).T
    selection_nums, annotations = columns[0], columns[-1]
    number_length = max([len(num_string) for num_string in selection_nums])

    # standardise number of zeroes in whistle selection numbers
    selection_nums = np.array([num_string.zfill(2) if len(num_string) else num_string for num_string in selection_nums])

    # use annotations column in selection table to group whistles
    for i, annotation in enumerate(annotations):
        if (annotation in ["W", "W_C", "W_NC"]):
            shutil.copy(selection_table[:-removed_end_str_length] + "sel_" + selection_nums[i] + ".wav", whistle_dir_name)
        elif annotation in ["NW"]:
            shutil.copy(selection_table[:-removed_end_str_length] + "sel_" + selection_nums[i] + ".wav", non_whistle_dir_name)
